In [3]:
#trainIDs = (pd.read_csv('shaply/trainidx_adrenal.csv')).to_numpy()

In [4]:
import numpy as np
import pandas as pd
import torch
from sklearn.preprocessing import StandardScaler

dataset=pd.read_csv('Datasets/adrenalTumorData_red.csv',header=None)
#dataset=pd.read_csv('shapley/ColoradoData_reduced.csv',header=None)
scaler = StandardScaler()


weight_set=False
#mtrx=pd.read_csv('shaply/omegaColorado_fold5.txt',header=None)
data= (dataset.iloc[:,:-1])#.to_numpy()
data = pd.DataFrame(scaler.fit_transform(data), columns=data.columns)

labels=(dataset.iloc[:,-1]).to_numpy()

feature_names=[names for names in dataset.columns.to_numpy()[:-1]]
data = data.apply(pd.to_numeric, errors='coerce',downcast='float')


input_data = torch.tensor(data.to_numpy())  # Batch size of 1 for simplicity
labels = torch.tensor(labels)
num_classes = np.unique(labels.numpy()).size
#print(input_data[0][0].dtype,model.fc.weight.dtype)

# Example dimensions
#input_dim = mtrx.shape[0]
input_dim = len(feature_names)
output_dim = num_classes

# Assuming these are your predefined weights (you need to replace these with your actual weights)
#init_weights_fc = torch.tensor(mtrx.to_numpy())
one_hot = torch.nn.functional.one_hot(labels).to(torch.float64)

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(input_data, labels, test_size = 0.2, random_state=1)

In [ ]:
def set_weight():
    #Set your custom initial weights
    with torch.no_grad():
        model.fc.weight.copy_(init_weights_fc)

In [7]:
import torch
import torch.nn as nn

# Define the structure of your single-layer model with softmax output
class SingleLayerSoftmax(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(SingleLayerSoftmax, self).__init__()
        self.fc = nn.Linear(input_dim, input_dim+50, bias=False)
        self.sigmoid = nn.Sigmoid()
        self.fc1 = nn.Linear(input_dim+50, output_dim,bias=False)
        self.softmax = nn.Softmax(dim=1)  # Apply softmax along the last dimension (dim=1)
        
    def forward(self, x):
        out = self.fc(x)
        #out = self.sigmoid(out)
        out = self.fc1(out)
        out = self.softmax(out)
        return out

# Initialize the model
model = SingleLayerSoftmax(input_dim, output_dim)
print(model)

# Example usage: forward pass with a random input
#model=model.double()


#for param in model.parameters():
#    param.data = param.data.to(torch.int64)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()  # Suitab for multi-class classification problems
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer with learning rate of 0.001

if weight_set:
    set_weight()

SingleLayerSoftmax(
  (fc): Linear(in_features=11, out_features=61, bias=False)
  (sigmoid): Sigmoid()
  (fc1): Linear(in_features=61, out_features=3, bias=False)
  (softmax): Softmax(dim=1)
)


In [8]:
def train_model(n,train_data,train_labels):
# Training loop
    for epoch in range(n):  # Train for 10 epochs
        optimizer.zero_grad()  # Zero the gradients
        outputs = model(train_data)  # Forward pass
        #print(outputs,train_labels)
   
        loss = criterion(outputs, train_labels)  # Calculate loss
            
        loss.backward()  # Backward pass to compute gradients        
        optimizer.step()  # Update model parameters using Adam
            
        #print(f'Epoch {epoch+1}, Loss: {loss.item()}')
    
    #print("Training completed.")

In [9]:
import torchmetrics
def test_model(test_data,test_labels):
    # Forward pass to get predictions (raw scores)
    predictions_raw = model(test_data)
    # Convert raw scores into class indices (argmax along dimension 1, which represents classes)
    _, predicted_classes = torch.max(predictions_raw, dim=1)
    
    accuracy_metric = torchmetrics.Accuracy(task='multiclass',num_classes=num_classes)
    accuracy = accuracy_metric(predicted_classes, test_labels)
    #print('test accuracy:')
    print(accuracy)

In [17]:
from sklearn.model_selection import KFold
    
def train_inFold(fold, epoch):
    if fold>1:
        kf = KFold(n_splits=fold, shuffle=True, random_state=1)
        trainID_fold=[]
        testID_fold=[]
        fold_datasets={}
        fold_labels={}
        #trainIDs = (pd.read_csv('shaply/trainidx_adrenal.csv',header=None)).to_numpy()
        #for train_IDs in trainIDs:
        #    cv_result = (model,data[train_IDs-2],labels[train_IDs-2],cv=3)
        for fold, (trainID, testID) in enumerate(kf.split(input_data)):
            #traintest_indices[fold]=[trainID,testID]
            trainID_fold.append(trainID)
            testID_fold.append(trainID)
            #print([input_data[i] for i in trainID])
            fold_datasets[fold]=[[input_data[i] for i in trainID],[input_data[i] for i in testID]]
            fold_labels[fold]=[[labels[i] for i in trainID],[labels[i] for i in testID]]
            #print(fold_labels)
        for fold in range(len(fold_datasets.keys())):
            #print(model(torch.stack(fold_datasets[fold][0])))
            #print(weight_layer1)
            train_model(epoch,torch.stack(fold_datasets[fold][0]),torch.stack(fold_labels[fold][0]))
            test_model(torch.stack(fold_datasets[fold][0]),torch.stack(fold_labels[fold][0]))
            weight_layer1 = model.fc.weight.detach().numpy()
            np.savetxt('output\WeightMatrix_AdrenalFold'+str(fold)+'.txt',weight_layer1, fmt= '%s',delimiter=',')
            x1_=shap_profile()
            np.savetxt('output\ShapeProfileFold'+str(fold)+'.txt',x1_, fmt= '%s',delimiter=',')
        pd.DataFrame(trainID_fold).to_csv('output\AdrenalFold5_trainIDs-wt',index=False, header=False)
        pd.DataFrame(trainID_fold).to_csv('output\AdrenalFold5_testIDs-wt',index=False, header=False)
        

    elif fold==1:
        #print(weight_layer1,mtrx,weight_layer2)
        train_model(epoch,X_train,y_train)
        test_model(X_valid,y_valid)

    #df_x = pd.DataFrame(trainID_fold)
    
    #np.savetxt('trainIDs_AdrenalFold2',trainID_fold, fmt= '%s',delimiter=',')
    #np.savetxt('testIDs_AdrenalFold2',testID_fold, fmt= '%s',delimiter=',')

#weight_layer1 = model.fc.weight.detach().numpy()
#weight_layer2 = model.fc1.weight.detach().numpy()

<>:27: SyntaxWarning: invalid escape sequence '\W'
<>:29: SyntaxWarning: invalid escape sequence '\S'
<>:30: SyntaxWarning: invalid escape sequence '\A'
<>:31: SyntaxWarning: invalid escape sequence '\A'
<>:27: SyntaxWarning: invalid escape sequence '\W'
<>:29: SyntaxWarning: invalid escape sequence '\S'
<>:30: SyntaxWarning: invalid escape sequence '\A'
<>:31: SyntaxWarning: invalid escape sequence '\A'
C:\Users\subhashree\AppData\Local\Temp\ipykernel_13404\755861978.py:27: SyntaxWarning: invalid escape sequence '\W'
  np.savetxt('output\WeightMatrix_AdrenalFold'+str(fold)+'.txt',weight_layer1, fmt= '%s',delimiter=',')
C:\Users\subhashree\AppData\Local\Temp\ipykernel_13404\755861978.py:29: SyntaxWarning: invalid escape sequence '\S'
  np.savetxt('output\ShapeProfileFold'+str(fold)+'.txt',x1_, fmt= '%s',delimiter=',')
C:\Users\subhashree\AppData\Local\Temp\ipykernel_13404\755861978.py:30: SyntaxWarning: invalid escape sequence '\A'
  pd.DataFrame(trainID_fold).to_csv('output\AdrenalFol

In [18]:
train_inFold(5,2500)

tensor(0.9255)
tensor(0.8936)
tensor(0.9149)
tensor(0.9043)
tensor(0.9202)


In [12]:
import itertools as it
def featureSet_Permtns(feature_i):
    feature_set = []
    feature_name = feature_names.copy()
    del feature_name[feature_i]
    for i in range(len(feature_name)+1):
        feature_set += it.combinations(feature_name,i)
    return feature_set

In [13]:
def featureSet_PermtnsValues(feature_set,n):
    instance_set = []
    label_set = []

    for i in (range(len(feature_set))):
        data1 = input_data[n].detach().numpy().copy()
        indx = []
        for j in range(len(feature_set[i])):
            #print(j)
            indx.append(feature_names.index(feature_set[i][j]))
        for k in range(len(feature_names)):
            if k not in indx:
                data1[k] = 0
                #print(k, data1)
        #print(i,data1)
        instance_set.append( data1)
        label_set.append(labels[n])
    return instance_set

In [14]:
def shap_value(feature_i,n):
    #pred_data=model(input_data[n])#.detach().numpy()
    featureSet = featureSet_Permtns(feature_i)

    featureSet_WithFeature = featureSet_PermtnsValues(featureSet,n)
    #permtns_pred=[]
    #permtns_pred_withFeature=[]
    sum_wof=0
    
     
    for i in range(len(featureSet)):
        featureSet_WithFeature[i][feature_i] = input_data[n][feature_i]
    
    #print('1',(featureSet_PermtnsValues(featureSet,n)),'2',featureSet_WithFeature)
    permtns_pred = model(torch.tensor(featureSet_PermtnsValues(featureSet,n))).detach()
    permtns_pred_withFeature = model(torch.tensor(featureSet_WithFeature)).detach()
    

    sum_wof = (criterion(permtns_pred, permtns_pred_withFeature))
        #print('1',len(permtns_pred), '2',len(permtns_pred_withFeature))
    #sum_wof=0
    #sum_wf=0
    #for i in range(len(permtns_pred)):
     #   sum_wof+=((criterion(permtns_pred[i], permtns_pred_withFeature[i])))
    #sum_=sum_wof/len(permtns_pred)
    #for i in range(len(permtns_pred_withFeature)):
        #sum_wf+=((criterion(permtns_pred_withFeature[n], permtns_pred_withFeature)))
    #sum_wf/=len(permtns_pred_withFeature)
    #sum_ = abs(abs(sum_wof)-abs(sum_wf))
    #permtns_pred = clf.cross_val_predict(featureSet_PermtnsValues(featureSet,n))
    #permtns_pred_withFeature = clf.cross_val_predict(featureSet_WithFeature)
    #count = 0
    sum_ = sum_wof/len(featureSet)
    
    #for i in range (len(permtns_pred)):
    #    if any(permtns_pred_withFeature[i], permtns_pred[i]):
    #        count+=1
    #sum_= abs(abs(permtns_pred_withFeature)-abs(permtns_pred))
    #if count !=0:
    #    sip_feature_i = sum_/count
    #else:
    #    sip_feature_i = sum_
    #return sip_feature_i
    return sum_

In [15]:
def shap_profile():
    shap=[0 for i in range(len(feature_names))]

    for i in range (len(feature_names)):
        for j in range (len(input_data)):
            shap[i]+=abs(shap_value(i,j))
        shap[i]=shap[i]/len(input_data)
    return(shap)

In [16]:
x1_=shap_profile()
#x_=[round((torch.norm(x1_[i])).detach().numpy()/(sum([torch.norm(x1_[i]) for i in range(len(x1_))])).detach().numpy(),3) for i in range(len(x1_))]
#y_ =[round(i/sum(np.sum(abs(weight_layer1.dot(weight_layer1.T)),axis=1)),3) for i in np.sum(abs(weight_layer1.dot(weight_layer1.T)),axis=1)]
#z_=np.array((abs(weight_layer1.dot(weight_layer1.T)).diagonal()))
#x_ = [round(x1_[i].detach().numpy()/sum(np.sum(x1_[i].detach().numpy())),3) for i in range(len(x1_))]
#y_ =[round(i/sum(np.sum(abs(weight_layer1.dot(weight_layer1.T)),axis=1)),3) for i in np.sum(abs(weight_layer1.dot(weight_layer1.T)),axis=1)]
#y_ =[round(i/sum(np.sum(abs(weight_layer1.dot(weight_layer2.T)),axis=1)),3) for i in np.sum(abs(weight_layer1.dot(weight_layer2.T)),axis=1)]

C:\Users\subhashree\AppData\Local\Temp\ipykernel_13404\1233899208.py:15: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_new.cpp:257.)
  permtns_pred = model(torch.tensor(featureSet_PermtnsValues(featureSet,n))).detach()


In [ ]:
x1_


In [ ]:
import matplotlib.pyplot as plt

plt.bar(range(len(x1_)),height=x1_)

In [ ]:
#x_=[round((torch.norm(x1_[i])).detach().numpy()/(sum([torch.norm(x1_[i]) for i in range(len(x1_))])).detach().numpy(),3) for i in range(len(x1_))]
#y_ =[round(i/sum(np.sum(abs(weight_layer1.dot(weight_layer1.T)),axis=1)),3) for i in np.sum(abs(weight_layer1.dot(weight_layer1.T)),axis=1)]
z_=(np.array((abs(weight_layer1.dot(weight_layer1.T))).diagonal()))/sum(np.array((abs(weight_layer1.dot(weight_layer1.T))).diagonal()))

In [ ]:
%pip install scipy

In [ ]:
x_sort = x_.copy()
x_sort.sort(reverse=True)
x_rank = [x_sort.index(i)+1 for i in x_]
y_sort = y_.copy()
y_sort.sort(reverse=True)
y_rank = [y_sort.index(i)+1 for i in y_]
#shapval_rank={}
dict_shap_rank= {i:x_sort.index(i)+1 for i in x_}
dict_wt_mat={i:y_sort.index(i)+1 for i in y_}

bar_labels=[str(i) + '\n rank\n sh#'+str(dict_shap_rank[x_[feature_names.index(i)]]) +'\nwt#'+ str(dict_wt_mat[y_[feature_names.index(i)]]) for i in feature_names ]
feature_ranks = [str(i) + ' rank      sh#'+str(dict_shap_rank[x_[feature_names.index(i)]]) +'       wt#'+ str(dict_wt_mat[y_[feature_names.index(i)]]) for i in feature_names ]

In [ ]:
from scipy import stats
res_SIP_CIP = stats.spearmanr(x_, y_,)
res_DIG_CIP = stats.spearmanr(z_, y_,)

In [ ]:
%pip install seaborn

In [ ]:
import seaborn as sns
# Sample data
cor_data = {
    'SIP': x_,
    'CIP': y_,
    'DIAG': z_
}

# Create a DataFrame
cor_df = pd.DataFrame(cor_data)

# Calculate Spearman correlation matrix
corr_matrix, _ = stats.spearmanr(cor_df)

# Convert to DataFrame for better visualization
corr_df = pd.DataFrame(corr_matrix, index=cor_df.columns, columns=cor_df.columns)

# Plot the heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(corr_df, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Spearman Correlation Matrix')
plt.show()

In [ ]:
feature_ranks

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

matplotlib.rc("xtick", labelsize="small")
matplotlib.rc("ytick", labelsize="small")
plot_values ={'shap':x_, 'weight matrix' :y_,'diag':z_}
x=np.arange(len(x_))
width = 0.25
multiplier = 0
fig, ax = plt.subplots(layout='constrained')

for attribute, measurement in plot_values.items():
    offset = width * multiplier
    rects = ax.bar(x + offset, measurement, width, label=attribute)
    ax.bar_label(rects, padding=3)
    multiplier += 1

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Weights & Shapley Values')
ax.set_title('Shapley vs Weight Matrix')
ax.set_xticks(x + width, bar_labels)
ax.legend(loc='upper left', ncols=3)
ax.set_ylim(0, max(max(y_),max(x_),max(z_))+0.2)

plt.show()

In [ ]:
(np.absolute([[3,0],[1,-2]]),np.absolute([[1,2],[3,1]]))

In [ ]:
X_train, X_test = X.iloc[train_index], X.iloc[test_index]
y_train, y_test = y.iloc[train_index], y.iloc[test_index]